In [ ]:
# Setup Environment - Common imports and configuration
import json
import pandas as pd
import numpy as np
import math
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add CSS for left-aligned math display
from IPython.core.display import HTML
display(HTML("""
<style>
    /* Left-align all MathJax displays */
    .MathJax_Display {
        text-align: left !important;
        margin: 1em 0 !important;
    }
    
    /* Left-align handcalcs output */
    .jp-RenderedHTMLCommon .MathJax_Display {
        text-align: left !important;
    }
    
    /* Ensure equation numbers (if any) stay on the right */
    .MathJax_Display .MathJax {
        text-align: left !important;
    }
</style>
"""))

# Helper functions for data formatting
def format_with_units(value, unit_str, precision=2):
    """Format a value with units for display."""
    if value is None or (isinstance(value, float) and math.isnan(value)):
        return "—"
    return f"{value:.{precision}f} {unit_str}"

def mg_to_meq(concentration_mg_l, ion):
    """Convert mg/L to meq/L using proper molecular weights."""
    MOLECULAR_WEIGHTS = {
        'Ca_2+': 40.078, 'Mg_2+': 24.305, 'Na_+': 22.990,
        'HCO3_-': 61.017, 'Cl_-': 35.453, 'SO4_2-': 96.06
    }
    VALENCES = {
        'Ca_2+': 2, 'Mg_2+': 2, 'Na_+': 1,
        'HCO3_-': 1, 'Cl_-': 1, 'SO4_2-': 2
    }
    
    if concentration_mg_l is None or math.isnan(concentration_mg_l):
        return None
    
    mw = MOLECULAR_WEIGHTS.get(ion)
    valence = VALENCES.get(ion)
    
    if not mw or not valence:
        return concentration_mg_l / 20  # Approximate
    
    equivalent_weight = mw / valence
    return concentration_mg_l / equivalent_weight

# Import handcalcs for equation rendering
try:
    from handcalcs import render
    %load_ext handcalcs.render
    # Configure handcalcs to left-align
    from handcalcs import config
    config.use_latex_left_align = True
except ImportError:
    print("handcalcs not available - calculations will show as regular code")
    render = None
except AttributeError:
    # config.use_latex_left_align might not exist in older versions
    pass

# Import forallpeople for units
try:
    import forallpeople as si
    si.environment('default', top_level=True)
except ImportError:
    si = None
    print("forallpeople not available - units will be handled manually")

# Import plotting libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10, 6)
mpl.rcParams['font.size'] = 11

try:
    import plotly.graph_objects as go
    import plotly.express as px
    USE_PLOTLY = True
except ImportError:
    USE_PLOTLY = False
    print("Plotly not available, using matplotlib")

print("Environment setup complete")
print(f"Project root: {project_root}")
print(f"Run ID: {run_id}")

In [ ]:
# Load simulation data
sim_data = simulation
design = design_inputs
resin_info = resin_metadata

# Extract key sections
performance = sim_data.get('performance', {})
ion_tracking = sim_data.get('ion_tracking', {})
mass_balance = sim_data.get('mass_balance', {})
economics = sim_data.get('economics', {})
breakthrough_data = sim_data.get('breakthrough_data', {})

print(f"Loaded simulation data for {resin_info['display_name']}")
print(f"Breakthrough at {performance.get('service_bv_to_target', 0):.1f} BV")